In [1]:
# from backend.dendron.out_go import process_file
# from langchain.document_loaders import PDFPlumberLoader
from backend.dendron.pdf import process_pdf

# Define the path to the file, and other parameters
file_path = "/mnt/d/ai/cognimesh/output/chalkidouPrioritysettingAchievingUniversal2016a.pdf"
chunk_size = 1000
chunk_overlap = 50

# Now call the process_file function
process_pdf(file_path=file_path,
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap)



2023-11-11 06:35:16,006 - backend.dendron.out_go - INFO - File with SHA1 25c2656183c4bea554e3abf0eed5c93afa5f30b9 has already been processed with doc_id 77ea4e00-de07-4264-adcf-5b793c7e9271.


PDF file located: /mnt/d/ai/cognimesh/output/chalkidouPrioritysettingAchievingUniversal2016a.pdf
PDF processing completed and content updated in database.


In [1]:
import uuid
from backend.dendron.pdf import retrieve_and_verify_pdf, convert_pdf_to_txt

doc_id = uuid.UUID("adedeeaa-c961-4edd-af98-2e181970a8fa")
file = retrieve_and_verify_pdf(doc_id)
convert_pdf_to_txt(file.content)

PDF file located: /mnt/d/ai/cognimesh/output/burnettFutureInpatientAnticoagulation2013.pdf


'JThrombThrombolysis(2013)35:375–386\nDOI10.1007/s11239-013-0892-1\nThe future of inpatient anticoagulation management\nAllison E. Burnett • Toby C. Trujillo\nPublishedonline:17February2013\n(cid:2)SpringerScience+BusinessMediaNewYork2013\nAbstract Anticoagulation therapy management has Past, present and future of anticoagulation\nevolved dramatically in both the outpatient and inpatient management\narenas over the last 30 years in the interest of optimizing\npatient safety and outcomes. With the advent of target In the late 1980s, there was an evolutionary shift in the\nspecificoralanticoagulantsandnextgenerationantiplatelet management of chronic oral anticoagulation, moving from\nagents, a new world of ‘‘antithrombosis management’’ is standard management by community providers to man-\nquickly becoming evident. Additionally, significant chan- agement by outpatient specialty anticoagulation clinics.\nges to the healthcare system in the United States are Factors that promoted this shif

In [18]:
from backend.axon.in_come import File
file = File()
file.import_path(filepath=file_path,
                    chunk_size=chunk_size,
                    chunk_overlap=chunk_overlap)

In [22]:
display(file.content)

b'%PDF-1.4\r%\xe2\xe3\xcf\xd3\r\n212 0 obj\r<</Linearized 1/L 335715/O 215/E 92424/N 12/T 331354/H [ 1956 526]>>\rendobj\r            \r\nxref\r\n212 83\r\n0000000016 00000 n\r\n0000002482 00000 n\r\n0000002650 00000 n\r\n0000002686 00000 n\r\n0000003129 00000 n\r\n0000003241 00000 n\r\n0000003353 00000 n\r\n0000003466 00000 n\r\n0000003579 00000 n\r\n0000003692 00000 n\r\n0000004330 00000 n\r\n0000004494 00000 n\r\n0000005391 00000 n\r\n0000005806 00000 n\r\n0000005843 00000 n\r\n0000005946 00000 n\r\n0000007116 00000 n\r\n0000008280 00000 n\r\n0000008352 00000 n\r\n0000021860 00000 n\r\n0000023181 00000 n\r\n0000023467 00000 n\r\n0000024303 00000 n\r\n0000025051 00000 n\r\n0000025192 00000 n\r\n0000025932 00000 n\r\n0000026721 00000 n\r\n0000027460 00000 n\r\n0000028268 00000 n\r\n0000028431 00000 n\r\n0000028731 00000 n\r\n0000028963 00000 n\r\n0000029766 00000 n\r\n0000030335 00000 n\r\n0000033028 00000 n\r\n0000043391 00000 n\r\n0000053849 00000 n\r\n0000057147 00000 n\r\n00000572

In [16]:
import uuid
from backend.neuron.cognition import Cognition
from backend.neuron.memory import Memory
from backend.neuron.intellect import Intellect
import numpy as np
from langchain.document_loaders import PDFPlumberLoader

# pg_doc_store = store # Assuming 'store' is your global Store instance
# Your provided doc_id as a UUID object
doc_id = uuid.UUID("adedeeaa-c961-4edd-af98-2e181970a8fa")
memory = Memory()
cognition = Cognition()
doc_embeddings = memory.retrieve_embeddings(doc_id)
query = 'From the content, please show me the all the researchers who contributred this article.'
query_embedding = cognition.embed_text(query)

intellect_instance = Intellect()
try:
    similarities = intellect_instance.calculate_similarity(doc_embeddings, query_embedding)
    findings = intellect_instance.present_findings(similarities)
    print(similarities)
except Exception as e:
    logger.error(f"An error occurred in similarity search: {e}")
    
chunk_index = np.argmax(similarities)
chunk = memory.retrieve_chunk(doc_id, chunk_index, PDFPlumberLoader)

[0.70797037 0.71816802 0.71388208 0.71237329 0.7176752  0.70299955
 0.69386322 0.71168873 0.71545698 0.726466   0.70554442 0.70550055
 0.70480334 0.70319646 0.7142309  0.7264558  0.72568243 0.71741544
 0.71944041 0.72026357 0.71992838 0.72548462]


In [17]:
chunk

Document(page_content='380 A.E.Burnett,T.C.Trujillo\nFig.1 Exampleofresource\nalgorithmforperi-operative\nanticoagulation\nregarding more readily available anticoagulant assays for anticoagulant agents will continue to come to market\ntarget specific agents. subsequent todabigatran, rivaroxaban, apixaban, prasugrel\nand ticagrelor, and our understanding of how to optimally\nManagement of patients experiencing bleeding with newer manage patients presenting with hemorrhagic complica-\nantithrombotic therapies The approach to managing tions will consistently and dramatically evolve.\nantithrombotic-related bleeding has received new focus This requires an expansion in the role of inpatient\nwith the advent of TSOACs and newer oral antiplatelet anticoagulation specialists from just managing anticoagu-\nagents. Management of bleeding in patients receiving tra- lation therapy to being more involved in the primary\nditional agents has, to some degree, become familiar and management of bleeding

In [ ]:
from backend.plugins.doi_handle import DOIHandle
# from ..settings.storage import store
from backend.settings.storage import store

doi_plugin = DOIHandle()
db_connection = store.connection

# You might want to catch any exceptions here and handle them appropriately
try:
    doi_plugin.setup(db_connection)
except Exception as e:
    # Log the exception, alert the user, etc.
    print(f"An error occurred during DOIHandle plugin setup: {e}")

try:
    # Disable the plugin first
    doi_plugin.switch(enable=False)
    
    # Then call the delete method to clean up resources
    doi_plugin.delete()

except Exception as e:
    # Log the exception, alert the user, etc.
    print(f"An error occurred during DOIHandle plugin deletion: {e}")


In [2]:
from backend.plugins.zotero_connect import ZoteroConnect
# from ..settings.storage import store
from backend.settings.storage import store

# Create an instance of the plugin
zotero_plugin = ZoteroConnect()

# Assuming the 'store' object is your database connection from backend_setting
db_connection = store.connection

# Enable the plugin (if your system requires this step)
zotero_plugin.switch(enable=True)

# Set up the plugin, which includes table creation
zotero_plugin.setup(db_connection)

# You might want to catch any exceptions here and handle them appropriately
try:
    zotero_plugin.setup(db_connection)
except Exception as e:
    # Log the exception, alert the user, etc.
    print(f"An error occurred during ZoteroConnect plugin setup: {e}")

try:
    # Disable the plugin first
    zotero_plugin.switch(enable=False)
    
    # Then call the delete method to clean up resources
    zotero_plugin.delete()

except Exception as e:
    # Log the exception, alert the user, etc.
    print(f"An error occurred during ZoteroConnect plugin deletion: {e}")

2023-11-10 21:04:47,435 - zotero_connect - INFO - ZoteroConnect plugin has been enabled.
2023-11-10 21:04:47,438 - zotero_connect - INFO - Setting up ZoteroConnect plugin.
2023-11-10 21:04:47,442 - zotero_connect - INFO - Creating zotero_articles and zotero_books tables.
2023-11-10 21:04:47,438 - zotero_connect - INFO - Setting up ZoteroConnect plugin.
2023-11-10 21:04:47,442 - zotero_connect - INFO - Creating zotero_articles and zotero_books tables.
2023-11-10 21:04:47,467 - zotero_connect - INFO - ZoteroConnect plugin has been enabled.
2023-11-10 21:04:47,467 - zotero_connect - INFO - ZoteroConnect plugin setup completed successfully.
2023-11-10 21:04:47,468 - zotero_connect - INFO - Setting up ZoteroConnect plugin.
2023-11-10 21:04:47,475 - zotero_connect - INFO - ZoteroConnect plugin has been enabled.
2023-11-10 21:04:47,476 - zotero_connect - INFO - ZoteroConnect plugin setup completed successfully.
2023-11-10 21:04:47,477 - zotero_connect - INFO - ZoteroConnect plugin has been di